In [1]:
import sys
sys.path.append("/home/meribejayson/Desktop/Projects/SharkCNN/training_models/YOLO3D/model_impl")
import yolo3d
import pynvml
import psutil

sys.path.append("/home/meribejayson/Desktop/Projects/SharkCNN/training_models/YOLO3D/yolo_dataloaders")

from predict_dataset import SharkYOLODataset
import cv2

import torch
import torch.utils.data as data

In [2]:
num_frames = 50
num_features = 16

torch.manual_seed(12)

if not torch.cuda.is_available():
    raise Exception("Couldn't find CUDA")

device = torch.device("cuda")

pynvml.nvmlInit()
handle = pynvml.nvmlDeviceGetHandleByIndex(0)

In [3]:
shark_dataset = SharkYOLODataset(num_frames=num_frames)
data_loader = data.DataLoader(shark_dataset, batch_size=1, num_workers=1, shuffle=True)

In [4]:
model = yolo3d.YOLO3D(num_frames=num_frames, num_features=num_features)

In [5]:
model = model.to(device=device)

In [6]:
item = shark_dataset.__getitem__(90)

item = item.to(device=device)

In [7]:

def print_system_memory_usage():
    memory = psutil.virtual_memory()
    print(f"System Memory Usage: {memory.percent}%")

def print_cpu_usage():
    print(f"CPU Usage: {psutil.cpu_percent()}%")

def print_gpu_memory_usage(handle):
    info = pynvml.nvmlDeviceGetMemoryInfo(handle)
    print(f"GPU Memory Usage: {info.used / info.total * 100:.2f}%")


In [8]:
# Before prediction
print_system_memory_usage()
print_cpu_usage()
print_gpu_memory_usage(handle)

# Perform the model prediction
model_output = model(item)

# After prediction
print_system_memory_usage()
print_cpu_usage()
print_gpu_memory_usage(handle)


System Memory Usage: 26.0%
CPU Usage: 9.8%
GPU Memory Usage: 17.77%
System Memory Usage: 26.3%
CPU Usage: 8.5%
GPU Memory Usage: 78.65%


In [9]:
low_preds, mid_preds, high_preds = model_output

In [10]:
print(f"Low level feature box map {low_preds[0].shape} Mid level feature box map {mid_preds[0].shape} High level feature box map {high_preds[0].shape}")

Low level feature box map torch.Size([1, 5, 68, 120]) Mid level feature box map torch.Size([1, 5, 34, 60]) High level feature box map torch.Size([1, 5, 17, 30])


In [11]:
print(f"Low level feature class map {low_preds[1].shape} Mid level feature class map {mid_preds[1].shape} High level feature class map {high_preds[1].shape}")

Low level feature class map torch.Size([1, 1, 68, 120]) Mid level feature class map torch.Size([1, 1, 34, 60]) High level feature class map torch.Size([1, 1, 17, 30])
